In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section c.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(446425, 19)


,hhcode,province,district,region,psu,hhno,idc,sec,scq01,scq02,scq03,scq04,scq05,scq06,scq07,scq08a,scq08b,scq09a,scq09b
234009,2.101002e+09,2,210,1,21010016,1,3,C,1.0,1.0,1,4.0,2.0,NaN,NaN,NaN,NaN,15.0,NaN
158173,1.302001e+09,1,130,2,13020006,2,2,C,1.0,1.0,1,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
264127,2.151001e+09,2,215,1,21510007,10,2,C,1.0,1.0,1,10.0,2.0,NaN,NaN,NaN,NaN,15.0,NaN
153945,1.292000e+09,1,129,2,12920003,3,10,C,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
122880,1.222002e+09,1,122,2,12220018,14,7,C,1.0,1.0,1,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Create a yes/no column for "ever admitted"
C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scq04"]
C["currently_enrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

In [3]:
# Reduce to the questions of interest (4 questions):

C_relevant = C[
    [
        "hhcode",
        "idc",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [4]:
C_relevant

,hhcode,idc,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province
0,1.001100e+09,1,2,NaN,NaN,6.0,1,1
1,1.001100e+09,2,2,NaN,NaN,5.0,1,1
2,1.001100e+09,3,2,NaN,NaN,NaN,1,1
3,1.001100e+09,1,1,5.0,2.0,NaN,1,1
4,1.001100e+09,2,2,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...
446420,4.262002e+09,3,2,NaN,NaN,6.0,2,4
446421,4.262002e+09,4,2,NaN,NaN,4.0,2,4
446422,4.262002e+09,5,2,NaN,NaN,4.0,2,4
446423,4.262002e+09,6,2,NaN,NaN,6.0,2,4


In [5]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2006_07_stata/section b.dta"
)
B.sample(5)

,hhcode,province,district,region,psu,section,idc,sbq01,sbq02,sbq03,sbq41,sbq42,sbq43,age,sbq05
175597,1.302001e+09,punjab,leiah,rural,13020013,B,4,male,present,son/daughter,1991,0,0,15,unmarried
7381,1.021001e+09,punjab,rawalpindi,urban,10210008,B,19,female,present,daughter in law/son in law,1986,0,0,20,married
189506,1.322004e+09,punjab,bahawalpur,rural,13220040,B,3,male,present,son/daughter,1982,0,0,24,unmarried
373792,3.122000e+09,nwfp,kark,rural,31220001,B,5,female,present,nephew/niece,1992,0,0,14,unmarried
339584,3.042002e+09,nwfp,chitral,rural,30420018,B,3,male,present,son/daughter,1995,0,0,10,unmarried


In [6]:
# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode','idc', 'age', 'sex', 'marital_status']].copy()

In [7]:
# Fix NAs
C_relevant['currently_enrolled'] = C_relevant['currently_enrolled'].fillna(value=str('no'))
C_relevant['currently_enrolled'].isna().value_counts()

False    446425
Name: currently_enrolled, dtype: int64

In [8]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [9]:
# MERGE
years06_07 = pd.merge(B_relevant, C_relevant, on=["hhcode", "idc"], how='inner', indicator=False)

# Evaluate merge 
#years06_07._merge.value_counts() #(ALL GOOD)

In [10]:
# Select people in school enrollment age (I would say 4-15 years old)
years06_07 = years06_07.loc[(years06_07["age"] >= 4)&(years06_07["age"] <= 15)]

In [11]:
years06_07.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    66049
2    42560
3    34792
4    29827
Name: province, dtype: int64

In [12]:
# Lets see which hh codes we have:
years06_07["hhcode"] = years06_07["hhcode"].astype("str")
years06_07['subprovince code']= years06_07['hhcode'].apply(lambda x: x[0:4])
years06_07["subprovince code"] = years06_07["subprovince code"].astype("int")

In [13]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0607_v1.csv"
)
strat_name.sample(5)

,province,subprovince,subprovince code long
67,NWFP,Chitral,30420
104,Balochistan,Panjgur,41720
14,Punjab,Attock,10120
80,NWFP,Abbottabad,31720
112,Balochistan,JhalMagsi,42520


In [14]:
strat_name['subprovince code long']= strat_name['subprovince code long'].astype("str")
strat_name['subprovince code']= strat_name['subprovince code long'].apply(lambda x: x[0:4])
strat_name['subprovince code'] = strat_name['subprovince code'].astype("int")
strat_name.sample(5)

,province,subprovince,subprovince code long,subprovince code
66,NWFP,LowerDir,30320,3032
5,Punjab,Lahore,11811,1181
51,Sindh,Nowshero Feroze,20420,2042
46,Punjab,Bahawalnager,13320,1332
93,Balochistan,Ziarat,40620,4062


In [15]:
# MERGE
years06_07_ = pd.merge(years06_07, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years06_07_._merge.value_counts()

both          144401
left_only          0
right_only         0
Name: _merge, dtype: int64

In [16]:
years06_07_.sample(3)

,hhcode,idc,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrolled,why_not,region,province_x,subprovince code,province_y,subprovince,subprovince code long,_merge
72368,2102000306.0,7,9,male,unmarried,1,0.0,1.0,NaN,2,2,2102,Sindh,Hyderabad,21020,both
51156,1332001405.0,9,4,female,unmarried,2,NaN,no,1.0,2,1,1332,Punjab,Bahawalnager,13320,both
136023,4182000715.0,3,8,male,unmarried,1,2.0,1.0,NaN,2,4,4182,Balochistan,Zhob,41820,both


In [17]:
# add year
years06_07_['year']=2005
years06_07_['marital_status']=years06_07_['marital_status'].astype('str')

#fix marital status categories
years06_07_.loc[(years06_07_['marital_status'] == "unmarried"), 'marital_status'] = 1
years06_07_.loc[(years06_07_['marital_status'] == "married"), 'marital_status'] = 2
years06_07_.loc[(years06_07_['marital_status'] == "only nikha has been solemanised,rukhsati not taken"), 'marital_status'] = 5
years06_07_.loc[(years06_07_['marital_status'] == "widow/widower"), 'marital_status'] = 3
years06_07_.loc[(years06_07_['marital_status'] == "divorced"), 'marital_status'] = 4
years06_07_['marital_status']=years06_07_['marital_status'].astype('int')

# fix column names
years06_07_['province']=years06_07_['province_y']

In [18]:
#select relevant columns
years06_07_ = years06_07_[
    [
        "hhcode",
        "idc",
        "age",
        "marital_status",
        "sex",
        "ever_admitted",
        "currently_enrolled",
        "region",
        "province",
        "subprovince code",
        "subprovince",
        "year",
    ]
].copy()

In [19]:
years06_07_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years06_07_.csv', index_label="id")

# PART 2

In [20]:
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("yes", 1)
years06_07_["currently_enrolled"] = years06_07_["currently_enrolled"].replace("no", 0)
years06_07_.sample(5)

,hhcode,idc,age,marital_status,sex,ever_admitted,currently_enrolled,region,province,subprovince code,subprovince,year
76700,2122002015.0,5,4,1,male,2,0.0,2,Sindh,2122,Thatta,2005
43476,1282001502.0,3,15,1,male,2,0.0,2,Punjab,1282,D.G. Khan,2005
108332,3132000601.0,5,13,1,male,1,1.0,2,NWFP,3132,Hangu,2005
143448,4252001616.0,4,7,1,female,2,0.0,2,Balochistan,4252,JhalMagsi,2005
30454,1192001706.0,3,9,1,male,1,1.0,2,Punjab,1192,Kasur,2005


In [21]:
# PRE-PROCESSING FOR DIFF-IN-DIFF DATA
# convert currently_enrolled from string to integer

# check region for anomalies

years06_07_["currently_enrolled"]= years06_07_["currently_enrolled"].astype('int').copy()
df_grp_1 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].count().reset_index()
df_grp_2 = years06_07_.groupby(["sex","subprovince","region"])["currently_enrolled"].sum().reset_index()

# merge data set
df_grp_merge = pd.merge(df_grp_1, df_grp_2, on=["sex","subprovince","region"], indicator=True)
# check merge
df_grp_merge._merge.value_counts()

both          408
left_only       0
right_only      0
Name: _merge, dtype: int64

In [22]:
df_grp_merge.isna().any()

sex                     False
subprovince             False
region                  False
currently_enrolled_x    False
currently_enrolled_y    False
_merge                  False
dtype: bool

In [23]:
# Code Cell 2:
# compute enrollment_rate
df_grp_merge["rate_enrollment"] = df_grp_merge["currently_enrolled_y"]/df_grp_merge["currently_enrolled_x"]

# rename columns
df_grp_merge = df_grp_merge.rename(columns={"currently_enrolled_x":"sample_population", "currently_enrolled_y":"enrolled_total"})

# drop "_merge" column
df_grp_merge.drop(["_merge"], axis=1, inplace=True)
df_grp_merge.sample(5)

,sex,subprovince,region,sample_population,enrolled_total,rate_enrollment
210,female,Badin,1,0,0,NaN
279,female,Karachi,2,446,249,0.558296
318,female,Malakand,1,0,0,NaN
7,male,Badin,2,845,487,0.576331
224,female,Bhawalpur,1,441,354,0.802721


In [25]:
df_grp_merge.to_csv("/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/20_analysis/enrollment_clean/years06_07_merge_diff.csv")

In [26]:
df_grp_merge.isna().any()

sex                  False
subprovince          False
region               False
sample_population    False
enrolled_total       False
rate_enrollment       True
dtype: bool